<a href="https://colab.research.google.com/github/nohwiin/ML/blob/master/Capstone2020/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
%cd "/gdrive/My Drive/Capstone2020/classification"

In [ ]:
! ls

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch


class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean


    def __call__(self, tensor):
        noise = tensor + torch.randn(tensor.size()) * self.std + self.mean
        return noise

    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

# 출처 https://discuss.pytorch.org/t/how-to-add-noise-to-mnist-dataset-when-using-pytorch/59745

In [ ]:
from torchvision import transforms
from torch.utils.data import DataLoader
from garbage_cls import dataset
from garbage_cls import data
from garbage_cls import GarbageCls
from torch import nn
from torchvision import models
import torchvision.transforms
from torchvision.transforms import CenterCrop, RandomHorizontalFlip, ToTensor, Normalize, RandomApply, RandomPerspective,RandomVerticalFlip,Resize


DATA_ROOT_DIR= '/gdrive/My Drive/Capstone2020/Dataset'

app = GarbageCls(DATA_ROOT_DIR, 'checkpoints')

hyper_params = {
    'num_epochs': 100,
    'lr': 0.001,
    'lr_gamma': 0.5,
    'lr_milestones': [80],
    'log_interval': 10,
    'checkpoint_interval': 1000
}
# batch size =128
    
app.set_hyper_params(**hyper_params)

model = models.densenet121(pretrained=True)

in_features = model.classifier.in_features
# 전체 freeze
for param in model.parameters():
    param.requires_grad_(False)
    pass

for param in model.features[-2].parameters():
  param.requires_grad_(True)
  pass

model.classifier = nn.Linear(in_features, app.num_classes)

model = model.cuda()
               
app.set_model(model)

app.set_data_transform('train', transforms.Compose(
                          [
                           Resize((300,300)),
                           RandomHorizontalFlip(),
                           RandomVerticalFlip(),
                           ToTensor(),
                           Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                           RandomApply([AddGaussianNoise(0., 0.1)], p=0.2)
                           ])
                      )
app.set_data_transform('val', transforms.Compose(
                        [
                         Resize((300,300)),
                         ToTensor(),
                         Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                        ])
                      )

app.run()

In [ ]:
from torchvision import transforms
from torch.utils.data import DataLoader
from garbage_cls import dataset
from garbage_cls import data
from garbage_cls import GarbageCls
from torch import nn
from torchvision import models
import torchvision.transforms
from torchvision.transforms import CenterCrop, RandomHorizontalFlip, ToTensor, Normalize, RandomApply, RandomPerspective,RandomVerticalFlip,ColorJitter,Resize

TEST_ROOT_DIR='/gdrive/My Drive/Capstone2020/Dataset'

app = GarbageCls(TEST_ROOT_DIR, 'checkpoints')

# DenseNet initializae
model = models.densenet121(pretrained=True)
in_features = model.classifier.in_features
for param in model.parameters():
    param.requires_grad_(False)
    pass

model.classifier = nn.Linear(in_features, app.num_classes)

model = model.cuda()
app.set_model(model)

app.set_data_transform('test', transforms.Compose([
                         Resize((300,300)),
                         ToTensor(),
                         Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
                      )

app.test()

In [ ]:
# model.cpu()
# model.eval()
# example = torch.rand(1, 3, 224, 224)
# traced_script_module = torch.jit.trace(model, example)
# traced_script_module.save("/gdrive/My Drive/model.pt")